In [6]:
import MySQLdb
import MySQLdb.cursors
import pandas as pd
from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression as LinR
import numpy as np
import sys
import re
%matplotlib inline
import matplotlib as plt
from __future__ import division
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

In [7]:
conn = MySQLdb.connect(
            user="root",
            passwd="",
            db="Player_Team_Data",
            cursorclass=MySQLdb.cursors.DictCursor)

In [8]:
player = 'james harden'
cmd_Rk= 'SELECT Rk FROM NBA_player_data WHERE Player_Name IN (\'' + player + '\')  AND Year IN (\'2015\');'
df_Rk = pd.read_sql(cmd_Rk, con=conn) 

def get_fanduel(predict,avg,tru):
    fanduel_pre = predict[0]+predict[2]*1.2+predict[3]*1.5+predict[4]*2+predict[5]*2-predict[6]
    fanduel_avg = avg[0]+avg[2]*1.2+avg[3]*1.5+avg[4]*2+avg[5]*2-avg[6]
    fanduel_tru = tru[0]+tru[2]*1.2+tru[3]*1.5+tru[4]*2+tru[5]*2-tru[6]
    return fanduel_pre, fanduel_avg, fanduel_tru

def get_trend(fan_pre,fan_avg,fan_tru):
    
    if (fan_pre > fan_avg) and (fan_tru > fan_avg):
        val = 1
    elif (fan_pre < fan_avg) and (fan_tru < fan_avg):
        val = 1
    elif (fan_pre < fan_avg) and (fan_tru > fan_avg):
        val = 0
    elif (fan_pre > fan_avg) and (fan_tru < fan_avg):    
        val = 0
    else:
        val = 2
    return val

In [9]:
f_pre = []
f_avg = []
f_tru = []

Rk = '22'
cmd_target_2015 = 'SELECT PTS,3P,TRB,AST,STL,BLK,TOV FROM NBA_player_data WHERE Player_Name IN (\'' + player + '\') AND Year IN (\'2015\') AND Rk < '+Rk+' ;'
cmd_target_2014 = 'SELECT PTS,3P,TRB,AST,STL,BLK,TOV FROM NBA_player_data WHERE Player_Name IN (\'' + player + '\') AND Year IN (\'2014\') AND Rk >= '+Rk+' ;'
cmd_train_2015 = 'SELECT Rk,Home_Away,DateDiff,TeamID,Win,OPPG,OTPR,O3Ppercent,ORPG,OBPG,OSPG,DEF,O3PM,OFGpercent,OTPG,OAPG,TPG,SPG,TRBR,OBLKpercent FROM NBA_player_data WHERE Player_Name IN (\'' + player + '\') AND Year IN (\'2015\') AND Rk < '+Rk+';'
cmd_train_2014 = 'SELECT Rk,Home_Away,DateDiff,TeamID,Win,OPPG,OTPR,O3Ppercent,ORPG,OBPG,OSPG,DEF,O3PM,OFGpercent,OTPG,OAPG,TPG,SPG,TRBR,OBLKpercent FROM NBA_player_data WHERE Player_Name IN (\'' + player + '\') AND Year IN (\'2014\') AND Rk >= '+Rk+';'
cmd_operate = 'SELECT Rk,Home_Away,DateDiff,TeamID,Win,OPPG,OTPR,O3Ppercent,ORPG,OBPG,OSPG,DEF,O3PM,OFGpercent,OTPG,OAPG,TPG,SPG,TRBR,OBLKpercent FROM NBA_player_data WHERE Player_Name IN (\'' + player + '\') AND Year IN (\'2015\') AND Rk = '+Rk+';'
cmd_truth = 'SELECT PTS,3P,TRB,AST,STL,BLK,TOV FROM NBA_player_data WHERE Player_Name IN (\'' + player + '\') AND Year IN (\'2015\') AND Rk = '+Rk+' ;'
cmd_min_2015 = 'SELECT MP FROM NBA_player_data WHERE Player_Name IN (\'' + player + '\') AND Year IN (\'2015\') AND Rk < '+Rk+';'   
cmd_min_2014 = 'SELECT MP FROM NBA_player_data WHERE Player_Name IN (\'' + player + '\') AND Year IN (\'2015\') AND Rk >= '+Rk+';'   

df_target_2015 = pd.read_sql(cmd_target_2015, con=conn) 
df_target_2014 = pd.read_sql(cmd_target_2014, con=conn)
df_min_2015 = pd.read_sql(cmd_min_2015, con=conn) 
df_min_2014 = pd.read_sql(cmd_min_2014, con=conn) 
df_train_2015 = pd.read_sql(cmd_train_2015, con=conn) 
df_train_2014 = pd.read_sql(cmd_train_2014, con=conn) 
df_operate = pd.read_sql(cmd_operate, con=conn) 
df_truth = pd.read_sql(cmd_truth, con=conn) 
df_truth = df_truth.applymap(lambda x: float(x))

df_min = pd.concat([df_min_2014, df_min_2015],ignore_index=True)
df_target=pd.concat([df_target_2014, df_target_2015],ignore_index=True)
df_train=pd.concat([df_train_2014, df_train_2015],ignore_index=True)
df_target = df_target.applymap(lambda x:float(x))
df_train = df_train.applymap(lambda x:float(x))
df_target_2015 = df_target_2015.applymap(lambda x: float(x))

df_inquire = df_operate.applymap(lambda x:float(x))
df_train_plus_inquire=pd.concat([df_train, df_inquire])
df_raw = df_train_plus_inquire.reindex()
df_raw_scaled = df_raw.copy()
df_raw_scaled = df_raw_scaled.applymap(lambda x: np.log(x))
df_raw_transform = df_raw.copy()

df_raw_scaled = df_raw_scaled.apply(lambda x:preprocessing.StandardScaler().fit(x).transform(x))
df_raw_transform = df_raw_transform.apply(lambda x:preprocessing.StandardScaler().fit(x))

TS = TSNE()
X=TS.fit_transform(df_raw_scaled.values)

#pca.n_components=6
#train_reduced = pca.fit_transform(df_raw_scaled)
#df_train_reduced=pd.DataFrame(train_reduced)
#     df_evaluate = df_raw_scaled.tail(1)
#     df_train_scaled = df_raw_scaled.iloc[:-1]
# df_evaluate = df_train_reduced.tail(1)
# df_train_scaled = df_train_reduced.iloc[:-1]

# #     rf = RandomForestRegressor(n_estimators=100)
# #     rf.fit(df_train_scaled, df_target)
# #     predictions = rf.predict(df_evaluate).round()[0]

# PTS = LinR()
# PTS.fit(df_train_scaled, df_target.PTS)
# pPTS = PTS.predict(df_evaluate)
# REB = LinR()
# REB.fit(df_train_scaled, df_target.TRB)
# pREB = REB.predict(df_evaluate)
# AST = LinR()
# AST.fit(df_train_scaled, df_target.AST)
# pAST = AST.predict(df_evaluate)
# TP = LinR()
# TP.fit(df_train_scaled, df_target['3P'])
# pTP = TP.predict(df_evaluate)
# STL = LinR()
# STL.fit(df_train_scaled, df_target.STL)
# pSTL = STL.predict(df_evaluate)
# BLK = LinR()
# BLK.fit(df_train_scaled, df_target.BLK)
# pBLK = BLK.predict(df_evaluate)
# TOV = LinR()
# TOV.fit(df_train_scaled, df_target.TOV)
# pTOV = TOV.predict(df_evaluate)

# predictions = np.asarray([pPTS,pTP,pREB,pAST,pSTL,pBLK,pTOV])

# if int(Rk) < 20:
#     average_stats=df_target.mean().round()   
# elif int(Rk) >= 20:
#     average_stats=df_target_2015.mean().round()

# average_stats=np.asarray(average_stats)
# true_stats = np.asarray(df_truth)[0]
# fanduel_pre, fanduel_avg, fanduel_tru = get_fanduel(predictions,average_stats,true_stats)
# f_pre.append(fanduel_pre)
# f_avg.append(fanduel_avg)
# f_tru.append(fanduel_tru)

df_raw_scaled

,Rk,Home_Away,DateDiff,TeamID,Win,OPPG,OTPR,O3Ppercent,ORPG,OBPG,OSPG,DEF,O3PM,OFGpercent,OTPG,OAPG,TPG,SPG,TRBR,OBLKpercent
0,-0.248640,0.987096,-1.494248,-0.620084,0.349783,-0.109672,-1.014649,-0.134296,-0.240743,-0.230728,0.717510,-0.689076,-0.828352,-0.939093,0.023753,-0.827626,1.360084,-0.185671,0.730348,-0.178047
1,-0.342338,-1.013072,0.047979,0.582585,-0.429899,0.292711,-0.468885,0.888308,0.203813,0.893634,-0.025900,0.027670,0.245419,-0.039514,-0.103068,0.467734,0.811398,0.327915,-0.445966,0.911699
2,-0.294470,0.987096,1.590206,0.777575,0.929271,-0.087128,-2.618809,-0.547803,-0.611720,-1.156964,0.354986,-0.054099,-2.344146,-0.150205,-1.328264,-1.684623,-0.313661,-1.829868,0.830131,-1.328892
3,-0.204682,0.987096,0.047979,0.837400,-0.512198,0.403224,0.685061,1.410248,0.203813,1.165919,-0.636038,0.620102,1.279683,0.614513,0.086530,0.661593,-0.742523,0.524414,-0.082212,1.145604
4,0.126431,-1.013072,0.047979,0.837400,-0.512198,0.403224,0.685061,1.410248,0.203813,1.165919,-0.636038,0.620102,1.279683,0.614513,0.086530,0.661593,-0.742523,0.524414,-0.082212,1.145604
5,-0.162447,-1.013072,0.950124,-1.677372,-0.029853,-1.843570,-1.398551,1.226006,-0.281636,1.603255,0.717510,-1.568422,-0.601657,-1.576914,-0.361934,-0.827626,1.058928,-1.447820,1.077904,1.679764
6,-0.121807,0.987096,0.047979,-0.341981,0.987871,-2.661866,-2.092904,-0.674209,-1.202362,-0.050227,-0.223830,-2.775658,-2.344146,-2.412994,0.210848,-2.549288,0.493246,-0.400760,1.225427,0.121565
7,-0.008350,0.987096,1.590206,0.894967,0.917892,-1.746366,-1.204476,-0.012526,-0.736873,-0.324009,-0.636038,-1.974808,-0.944176,-1.343089,0.086530,-0.938124,-0.383891,0.327915,-0.030665,-0.417420
8,0.026959,-1.013072,-1.494248,-0.001610,0.113860,-0.773412,0.044565,0.581329,-1.722416,0.369408,-0.960731,0.298516,-0.061734,0.452606,-0.427774,-0.993790,-0.597546,-0.734703,0.680310,0.534892
9,0.061148,-1.013072,0.047979,0.358964,0.304243,0.314857,0.529316,0.581329,0.043260,1.165919,-0.223830,0.406123,0.637291,1.041201,0.633512,0.320074,-1.038842,1.353143,0.020777,0.971388


In [10]:
score =[]
for i,f in enumerate(f_pre):
    s=get_trend(f,f_avg[i],f_tru[i])
    score.append(s)


In [11]:
x = [1,2,3]
plt.pyplot.bar( x,[predictions[0],average_stats[0] , true_stats[0]])

NameError: name 'predictions' is not defined

In [ ]:
a = [x for x in score if x != 2]

In [ ]:
a.count(1)/len(a)



In [ ]:
def get_min(x):
    match = re.search('(\w+):(\w+)', x)
    minute = float(match.group(1))
    second = float(match.group(2))/60
    timet = minute+second
    return timet

In [ ]:
df_minx = df_min.applymap(lambda x: get_min(x))
df_minx

In [ ]:
df_minx.std()